##### Create dictionary of labels:urchan from Channels.csv

In [5]:
import pandas as pd

df = pd.DataFrame(pd.read_csv("Channels.csv"))

channels_dict=df.set_index('labels')['urchan']
channels_dict

labels
Fp1     1
Fp2     2
F3      3
F4      4
C3      5
       ..
PO8    63
Fpz    64
CPz    65
POz    66
Oz     67
Name: urchan, Length: 66, dtype: int64

##### Actual code

In [6]:
import mne
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import cupy

In [7]:
data_dir = Path("data")

# Initialize lists for data and labels
data_all = []
labels_all = []

In [8]:
#MNE param
# sfreq = 500.0  # Sampling frequency, adjust if known
# n_channels = channels_dict.values
# channel_names = list(channels_dict.keys())
# print(n_channels)
# print(channel_names)
# info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types="eeg")
os.environ['MNE_USE_CUDA'] = 'true' 
mne.utils.set_config('MNE_USE_CUDA', 'true') 

In [9]:
for file_path in data_dir.glob("*.set"):
    #Get the filename and determine the label based on the number at the beginning
    file_name = file_path.stem
    if "PREP" not in file_name:
        number = int(file_name.split('_')[0])
        label = 1 if number % 2 == 1 else 0  #1=Parkinson's 
                                             #0=Non-Parkinson's

        #Open .set file with MNE
        raw = mne.io.read_raw_eeglab(file_path, preload=True)
        
        #Bandpass filter to only keep signals between 1-30 Hz
        raw.filter(1., 30., fir_design='firwin', n_jobs='cuda')
        
        #Apply ICA to remove artifacts
        ica = mne.preprocessing.ICA(n_components=15, random_state=22, max_iter="auto")
        ica.fit(raw)
        raw = ica.apply(raw)

        # Convert to numpy format
        data = raw.get_data()  #Format:(n_channels, n_times)
        data = data.T  #Transpose

        #Append data and labels to create one large dataset
        data_all.append(data)
        labels_all.extend([label] * len(data))

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Now using CUDA device 0
Enabling CUDA with 10.83 GB available memory
Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 1.6s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 66 PCA components


C:\Users\Admin\AppData\Local\Temp\ipykernel_14068\3776364355.py:17: RuntimeWarning: Using n_components=15 (resulting in n_components_=15) may lead to an unstable mixing matrix estimation because the ratio between the largest (59) and smallest (8.5e-06) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 8
  ica.fit(raw)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 0.4s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 66 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
------

e:\Penn State Homework\Homework\CMPSC\CMPSC 445 Proj\.venv\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 0.4s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 66 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
------

...

e:\Penn State Homework\Homework\CMPSC\CMPSC 445 Proj\.venv\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 8.8s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 66 PCA components


e:\Penn State Homework\Homework\CMPSC\CMPSC 445 Proj\.venv\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 0.5s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 66 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
------

e:\Penn State Homework\Homework\CMPSC\CMPSC 445 Proj\.venv\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_all = torch.tensor(data_all,device=device)
labels_all = torch.tensor(labels_all, dtype=torch.long, device=device)

mean = torch.mean(data_all, dim=0, keepdim=True)
std = torch.std(data_all, dim=0, keepdim=True)
data_all = (data_all - mean) / std

X_combined = data_all
y_combined = labels_all


from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=42
)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_14068\3480506170.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  data_all = torch.tensor(data_all,device=device)


Using device: cuda


ValueError: expected sequence of length 79081 at dim 1 (got 77764)

In [ ]:
def train(model, train_dataloader, optimizer, print_freq=10):
    model.train()

    train_loss = 0
    
    for batch_index, (data, target) in enumerate(train_dataloader):
        # Move data and target to the same device as the model
        data, target = data.cuda(), target.cuda()  
        
        optimizer.zero_grad()
        output = model(data)

        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()

        optimizer.step()
        
        train_loss += loss.item() * data.shape[0]
        
        if not (batch_index % print_freq):
            print("Current Loss:", loss)

    return train_loss / len(train_dataloader.dataset)

def test(model, test_dataloader):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_index, (data, target) in enumerate(test_dataloader):
            # Move data and target to the same device as the model
            data, target = data.cuda(), target.cuda()
            
            output = model(data)
            
            test_loss += torch.nn.functional.nll_loss(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = correct / len(test_dataloader.dataset)
    
    return test_loss, test_accuracy

In [ ]:
def train_model(model, train_dataloader, test_dataloader, optimizer, num_epochs):
    loss_results=[]
    accuracy_results=[]
    for i in range(num_epochs):
        train_loss = train(model, train_dataloader, optimizer)
        test_loss, test_accuracy = test(model, test_dataloader)
        
        print(
            f'Epoch: {i + 1} | Train loss: {train_loss:.5f} |',
            f'Test loss: {test_loss:.5f} | Test accuracy: {test_accuracy:.5f}'
        )
        accuracy_results.append([i+1,test_accuracy])
        loss_results.append([i+1,test_loss])
    return accuracy_results,loss_results

In [ ]:
import torch.nn as nn

# Define the model
class EEGClassifier(nn.Module):
    def __init__(self):
        super(EEGClassifier, self).__init__()
        self.fc1 = nn.Linear(66, 64)  # Updated input size to match 66 channels
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = EEGClassifier().to(device)  # Move model to the appropriate device
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

accuracy_results,loss_results = train_model(model=model,train_dataloader=train_loader,test_dataloader=test_loader,optimizer=optimizer,num_epochs=10)
# Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     for X_batch, y_batch in dataloader:
#         # Move batch data to the same device as the model
#         X_batch = X_batch.to(device)
#         y_batch = y_batch.to(device)
        
#         optimizer.zero_grad()
#         outputs = model(X_batch)
#         loss = criterion(outputs, y_batch)
#         loss.backward()
#         optimizer.step()
    
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Using device: cuda
Epoch [1/10], Loss: 0.3482


KeyboardInterrupt: 